## 0. 연동 설정 

In [1]:
import sys
import os

# 현재 파일의 절대 경로
current_path = os.getcwd()

# 프로젝트의 루트 디렉토리 (현재 경로의 상위 디렉토리)
project_root = os.path.abspath(os.path.join(current_path, ".."))
# 마지막 디렉터리 이름을 추출
last_directory_name = os.path.basename(project_root)

# PYTHONPATH에 추가
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"현재 실행파일 경로: {current_path}")
print(f"프로젝트 루트 경로: {project_root}")
print(f"Python 경로: {sys.path}")
print("마지막 디렉토리 이름:", last_directory_name)

현재 실행파일 경로: /Users/jisu/Desktop/Capstone_NaverCloud/development_finbot_12_02/langchain
프로젝트 루트 경로: /Users/jisu/Desktop/Capstone_NaverCloud/development_finbot_12_02
Python 경로: ['/Users/jisu/Desktop/Capstone_NaverCloud/development_finbot_12_02', '/Users/jisu/miniconda/envs/lc/lib/python312.zip', '/Users/jisu/miniconda/envs/lc/lib/python3.12', '/Users/jisu/miniconda/envs/lc/lib/python3.12/lib-dynload', '', '/Users/jisu/miniconda/envs/lc/lib/python3.12/site-packages', '/Users/jisu/miniconda/envs/lc/lib/python3.12/site-packages/setuptools/_vendor']
마지막 디렉토리 이름: development_finbot_12_02


In [2]:
from config.api_config import (
    CLOVA_API_KEY, 
    CLOVA_API_GATEWAY_KEY
)

os.environ["NCP_CLOVASTUDIO_API_KEY"] = CLOVA_API_KEY
os.environ["NCP_APIGW_API_KEY"] = CLOVA_API_GATEWAY_KEY

## 1-2. LLM 체인(LLM Chain) 만들기

### 1-2-1. 기본 LLM 체인 (Prompt+LLM)

In [3]:
from langchain_community.chat_models import ChatClovaX

# model
llm = ChatClovaX(model="HCX-003")

# chain 실행
llm.invoke("자기자본수익률의 계산 방법은?")

# from langchain_openai import ChatOpenAI

# # model
# llm = ChatOpenAI(model='gpt-40-mini')

# # chain 실행
# llm.invoke("지구의 자전 주기는?")

AIMessage(content='자기자본수익률(ROE)은 기업이 자기자본을 활용하여 얼마만큼의 이익을 냈는지를 나타내는 지표로, 다음과 같이 계산됩니다.\n\n- ROE = (당기순이익 / 평균자기자본) x 100\n\n여기서 당기순이익은 해당 기간 동안 기업이 벌어들인 순이익을 의미하며, 평균자기자본은 기초와 기말의 자기자본을 평균한 값을 말합니다.\n\n예를 들어, A기업의 당기순이익이 1억 원', additional_kwargs={}, response_metadata={'stop_reason': 'length', 'input_length': 11, 'output_length': 101, 'seed': 1901473077, 'ai_filter': None}, id='run-6c18f598-1e83-481d-bc67-e0f18363925b-0', usage_metadata={'input_tokens': 11, 'output_tokens': 101, 'total_tokens': 112})

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate

# ChatPromptTemplate.from_template(): 문자열 형태의 템플릿을 인자로 받아, 해당 형식에 맞는 프롬프트 객체를 생성
# method1
prompt = ChatPromptTemplate.from_template("당신은 재무제표 가이드입니다. 다음 질문에 답해주세요. <Question>: {input}")
print(prompt)

# method2
# ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="당신은 재무제표 가이드입니다. 다음 질문에 답해주세요. <Question>: {input}"))])

input_variables=['input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='당신은 재무제표 가이드입니다. 다음 질문에 답해주세요. <Question>: {input}'), additional_kwargs={})]


In [5]:
# 다음의 코드를 통해, prompt와 LLM을 연결하여 chain을 구성한다. 이 chain을 이용하여 입력된 질문 "지구의 자전 주기는?"에 대한 답변을 생성하는 과정을 구현한다.

# model
llm = ChatClovaX(model="HCX-003")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "자기자본수익률의 계산 방법은?"})

AIMessage(content='자기자본수익률(ROE)은 기업이 투자한 자본 대비 얼마만큼의 이익을 창출했는지를 나타내는 지표로, 다음과 같이 계산됩니다.\n\n- ROE = 순이익 / 자기자본\n\n여기서 순이익은 기업이 일정 기간 동안 벌어들인 총이익에서 비용을 제외한 금액을 말합니다. 그리고 자기자본은 기업의 총자산에서 부채를 제외한 순수한 자본을 의미합니다.\n\n예를 들어, A기업의 순이익이 1억 원이고', additional_kwargs={}, response_metadata={'stop_reason': 'length', 'input_length': 26, 'output_length': 101, 'seed': 294560164, 'ai_filter': None}, id='run-b26985ae-47c6-431e-bf04-6096054f191d-0', usage_metadata={'input_tokens': 26, 'output_tokens': 101, 'total_tokens': 127})

In [6]:
# 다음과 같이 프롬프트, LLM, 문자열 출력 파서(StrOutParser)를 연결하여 체인을 만들 수 있다.

from langchain_community.chat_models import ChatClovaX
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("당신은 재무제표 가이드입니다. 다음 질문에 답해주세요. <Question>: {input}")
llm = ChatClovaX(model="HCX-003")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
result = chain.invoke({"input": "자기자본수익률의 계산 방법은?"})
print(result)

자기자본수익률(ROE)은 기업이 투자한 자본 대비 얼마만큼의 이익을 내고 있는지를 나타내는 지표로, 다음과 같은 공식으로 계산됩니다.

- ROE = (당기순이익 / 평균자기자본) x 100

여기서 당기순이익은 해당 기간 동안 기업이 벌어들인 순이익을 의미하며, 평균자기자본은 기초와 기말의 자기자본을 평균한 값을 말합니다.

예를 들어, A기업의 2021년 당기순이익이


### 1-2-3. 체인을 실행하는 방법

- LangChain의 "Runnable" 프로토콜은 사용자가 사용자 정의 체인을 쉽게 생성하고 관리할 수 있도록 설계된 핵심적인 개념이다. <br>
  이 프로토콜을 통해, 개발자는 일관된 인터페이스를 사용하여 다양한 타입의 컴포넌트를 조합하고, 복잡한 데이터 처리 파이프라인을 구성할 수 있다.
  "Runnable" 프로토콜은 다음과 같은 주요 메소드를 제공한다.
  - **invoke**: 주어진 입력에 대해 체인을 호출하고, 결과를 반환한다. 이 메소드는 단일 입력에 대해 동기적으로 작동한다.
  - **batch**: 입력 리스트에 대해 체인을 호출하고, 각 입력에 대한 결과를 '리스트'로 반환한다. 이 메소드는 여러 입력에 대해 동기적으로 작동하며, 효율적인 배치 처리를 가능하게 한다.
  - **stream**: 입력에 대해 체인을 호출하고, 결과의 조각들을 스트리밍한다. 이는 대용량 데이터 처리나 실시간 데이터 처리에 유용하다.
  - **비동기 버전**: ainvoke, abatch, astream 등의 메소드는 각각의 동기 버전에 대한 비동기 실행을 지원한다. 이를 통해, 비동기 프로그래밍 패러다임을 사용하여 더 높은 처리 성능과 효율을 달성할 수 있다.
    (이 때, 비동기 실행이란? 서버 컴퓨터의 작업이 끝날 때까지 기다리지 않는 통신으로, 서버에 요청(등록, 수정, 삭제 등)이 저장될 때 까지 기다리지 않고 다른 작업을 진행한다. <br>
     반면에, 동기 실행이란? 서버 컴퓨터의 작업이 끝날 때까지 기다린 후 다음 작업을 실행하는 통신이다.)

1. Invoke - 단일 입력에 대한 결과 반환

In [7]:
# Invoke
from langchain_community.chat_models import ChatClovaX
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("재무이론에서 {topic}에 대해 간단히 설명해주세요.")
llm = ChatClovaX(model="HCX-003")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
result = chain.invoke({"topic": "재무이론"})
print("[Invoke 결과]:", result)

[Invoke 결과]: 재무이론은 기업이나 개인이 자금을 조달하고 운용하는 데 필요한 이론으로, 다음과 같은 내용을 포함합니다.

1.**자금조달**: 기업이나 개인이 자금을 조달하는 방법에는 자기자본과 타인자본이 있습니다. 자기자본은 주식이나 채권 등을 발행하여 조달하며, 타인자본은 은행 대출이나 사채 등을 통해 조달합니다.

2.**투자결정**: 기업이나 개인이 투자를 결정할 때는 수익성과 위험성을


2. Batch: 각 입력에 대한 결과를 '리스트'로 반환

In [8]:
# Batch
topics = ["주당순이익", "이자보상배율", "자기자본수익률"]
results = chain.batch([{"topic": t} for t in topics])
print("[Batch 결과]:")
for topic, result in zip(topics, results):
    print(f"** {topic} 설명: {result}\n")

[Batch 결과]:
** 주당순이익 설명: 주당순이익(EPS)은 기업이 벌어들인 순이익을 그 기업이 발행한 총 주식 수로 나눈 값으로, 1주당 이익을 얼마나 창출했는지를 나타내는 지표입니다. 

주당순이익은 기업의 수익성을 측정하는 중요한 지표 중 하나로, 주당순이익이 높을수록 주식의 투자 가치가 높다고 볼 수 있습니다. 또한, 주당순이익은 주가수익비율(PER) 계산에도 활용되며, PER은 주가를 주당순이익으로 나눈 값으로, 해당 기업

** 이자보상배율 설명: 이자보상배율(Interest Coverage Ratio)은 기업이 영업활동으로 벌어들인 이익으로 이자비용을 얼마나 감당할 수 있는지를 나타내는 지표입니다. 

이자보상배율은 다음과 같이 계산됩니다.

이자보상배율 = 영업이익 / 이자비용

이자보상배율이 1보다 작으면, 기업이 영업활동으로 벌어들인 이익으로 이자비용을 감당하지 못한다는 것을 의미합니다. 이는 기업의 재무상태가 불안정하다는 것

** 자기자본수익률 설명: 자기자본수익률(ROE, Return on Equity)은 기업이 투자한 자본 대비 얼마만큼의 이익을 창출했는지를 나타내는 지표입니다. 

이는 기업의 경영성과를 평가하는 데 중요한 역할을 하며, 다음과 같이 계산됩니다.

- ROE = (당기순이익 / 평균자기자본) x 100

여기서 당기순이익은 기업이 일정 기간 동안 벌어들인 모든 수익에서 비용을 차감한 금액이며, 평균자기자본은 기업의



3. Stream: 결과의 조각들을 스트리밍. 결과가 조각조각 출력됨.

In [9]:
# Stream
stream = chain.stream({"topic": "주가예측"})
print("[Stream 결과]:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

[Stream 결과]:
주가예측은 재무이론에서 매우 중요한 분야 중 하나입니다. 주식시장에서는 다양한 요인들이 주가에 영향을 미치기 때문에, 정확한 예측을 하기 위해서는 많은 노력이 필요합니다.

일반적으로 주가를 예측하기 위해서는 다음과 같은 방법들이 사용됩니다.

1.**기본적 분석**: 기업의 재무상태, 경영성과, 산업 동향 등을 분석하여 주가를 예측하는 방법입니다. 이 방법은 기업의 내재가치를 파악


## 2. RAG 기법

### 2-2. RAG - DocumentLoader

2-2-5. PDF 문서 - PDFMiner 이용

In [10]:
!pip install pdfminer.six

In [11]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

pdf_folder = '/Users/jisu/Desktop/fd/analysist_report'

def extract_text_by_page(pdf_path):
    for page_layout in extract_pages(pdf_path):
        page_text = ""
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                page_text += element.get_text()
        yield page_text

# 폴더 내의 모든 PDF 파일 처리
for file_name in os.listdir(pdf_folder):
    if file_name.endswith('.pdf'):  # PDF 파일만 선택
        pdf_path = os.path.join(pdf_folder, file_name)
        print(f"** Processing file: {pdf_path}")
        for page_number, page_text in enumerate(extract_text_by_page(pdf_path), start=1):
            print(f"[Page {page_number}]:\n{page_text}\n{'-'*40}\n")


** Processing file: /Users/jisu/Desktop/fd/analysist_report/BGF리테일_2024.pdf
[Page 1]:
BGF 리테일 
(282330.KS) 
1Q24 Review:  점포  확장  효과와  고
정비  증가  간  시차 
Korea | 유통 | 2024.05.03 
투자의견 
BUY(유지) 
목표주가 
200,000 원(유지) 
현재주가 
133,100 원(05/02) 
시가총액 
2,300 (십억원) 
유통/패션 이해니_ 02)368-6155_ hnlee@eugenefn.com 
  1Q24P 매출액 1.95조원(+5.6%, 이하 yoy), 영업이익 326억원(-11.9%) 기록 
  동일점 성장률은 +0.6% 기록.  전년 높은 기저(+5.3%)에 비우호적인 기상 환경, 해외 출국자 증가 영향   
  영업이익 하락 주요 요인: 매출 성장률이 본부 임차형 증가 관련 고정 비용(사용권, 자산상각비)을 상쇄하지 못 함 
  연간 개점 목표(800점 유지) 순조롭게 달성 중임에도 평균 가맹 수수료율은 35.1%(+0.1%p)로 높은 레벨 지속 
  비중: 가공식품 42.6%(+0.5%p), 식품 13.6%(flat), 담배 8.1%(-0.4%p), 비식품 5.7%(-0.1%p) 
  성장 전략 1) 간편식 전면 리뉴얼 및 협업 상품 확대, 2) 디저트 히트 상품 발굴, 3) 주류 연계 구매 강화 
  비용 전략: 1) 코로나 이후 임차료 인상 러쉬 일단락 및 향후  임차료 협상 우위 노력, 2) 본부 임차형 개점 마무리 
  목표주가 200,000원 유지, 투자의견 ‘매수’ 유지 
12 월 결산(십억원) 
2022A  2023A  2024E  2025E 
매출액 
영업이익 
세전손익 
당기순이익 
EPS(원) 
증감률(%) 
PER(배) 
ROE(%) 
PBR(배) 
EV/EBITDA(배) 
자료: 유진투자증권 
7,616   8,195   8,683   9,154  
252  
254  


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.


[Page 4]:
BGF 리테일 
Compliance Notice 
당사는 자료 작성일 기준으로 지난 3 개월 간 해당종목에 대해서 유가증권 발행에 참여한 적이 없습니다 
당사는 본 자료 발간일을 기준으로 해당종목의 주식을 1% 이상 보유하고 있지 않습니다  
당사는 동 자료를 기관투자가 또는 제 3 자에게 사전 제공한 사실이 없습니다 
조사분석담당자는 자료작성일 현재 동 종목과 관련하여 재산적 이해관계가 없습니다 
동 자료에 게재된 내용들은 조사분석담당자 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭 없이 작성되었음을 확인합니다 
동 자료는 당사의 제작물로서 모든 저작권은 당사에게 있습니다 
동 자료는 당사의 동의 없이 어떠한 경우에도 어떠한 형태로든 복제, 배포, 전송, 변형, 대여할 수 없습니다 
동 자료에 수록된 내용은 당사 리서치센터가 신뢰할 만한 자료 및 정보로부터 얻어진 것이나, 당사는 그 정확성이나 완전성을 보장할 수 없습니다. 
따라서 어떠한 경우에도 자료는 고객의 주식투자의 결과에 대한 법적 책임소재에 대한 증빙자료로 사용될 수 없습니다 
투자기간 및 투자등급/투자의견 비율 
종목추천 및 업종추천 투자기간: 12 개월  (추천기준일 종가대비 추천종목의 예상 목표수익률을 의미함) 
ㆍSTRONG BUY(매수) 
ㆍBUY(매수) 
ㆍHOLD(중립) 
ㆍREDUCE(매도) 
추천기준일 종가대비 +50%이상 
추천기준일 종가대비 +15%이상 ~ +50%미만 
추천기준일 종가대비 -10%이상 ∼ +15%미만 
추천기준일 종가대비 -10%미만 
당사 투자의견 비율(%) 
1% 
93% 
5% 
1% 
(2024.03.31 기준) 
BGF리테일(282330.KS) 주가 및 목표주가 추이 
담당 애널리스트: 이해니 
과거 2년간 투자의견 및 목표주가 변동내역 
추천일자 
투자의견  목표가(원) 
2022-05-26 
2022-07-25 
2022-09-07 
2022-10-13 
2022-10-13 
2022-11-04 
2022-1

### 2-3. RAG - Text Splitter

In [12]:
import os
import logging
from typing import List, Generator
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from langchain.text_splitter import RecursiveCharacterTextSplitter

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def extract_text_by_page(pdf_path: str) -> Generator[str, None, None]:
    """PDF 파일에서 페이지별로 텍스트 추출"""
    try:
        for page_layout in extract_pages(pdf_path):
            page_text = ""
            for element in page_layout:
                if isinstance(element, LTTextContainer):
                    page_text += element.get_text()
            yield page_text.strip()
    except Exception as e:
        logger.error(f"텍스트 추출 중 오류 발생: {str(e)}")
        raise

def create_text_splitter() -> RecursiveCharacterTextSplitter:
    """청크 분할기 생성"""
    return RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " ", ""],
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

def process_pdf_chunks(pdf_folder: str):
    """PDF 파일들을 처리하고 청킹"""
    text_splitter = create_text_splitter()
    
    # PDF 파일 목록 가져오기
    pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    total_files = len(pdf_files)
    logger.info(f"총 {total_files}개의 PDF 파일을 처리합니다.")
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)
        logger.info(f"처리 중인 파일: {pdf_file}")
        
        try:
            # 페이지별로 텍스트 추출 및 청킹
            for page_num, page_text in enumerate(extract_text_by_page(pdf_path), start=1):
                if not page_text.strip():
                    continue
                    
                # 텍스트 청킹
                chunks = text_splitter.split_text(page_text)
                
                # 청크 출력
                logger.info(f"[Page {page_num}] - 총 {len(chunks)}개 청크 생성됨")
                for i, chunk in enumerate(chunks, 1):
                    logger.info(f"Chunk {i}: 길이 {len(chunk)} 문자")
                    # logger.info(f"내용: {chunk[:200]}...")  # 처음 200자만 출력
                logger.info("-" * 80)
                
        except Exception as e:
            logger.error(f"파일 처리 실패 ({pdf_file}): {str(e)}")
            continue

if __name__ == "__main__":
    pdf_folder = '/Users/jisu/Desktop/fd/analysist_report'
    
    try:
        process_pdf_chunks(pdf_folder)
        logger.info("\n모든 PDF 파일의 처리가 완료되었습니다.")
    except Exception as e:
        logger.error(f"처리 중 오류 발생: {str(e)}")

2024-12-02 20:43:54,273 - INFO - 총 2개의 PDF 파일을 처리합니다.
2024-12-02 20:43:54,273 - INFO - 처리 중인 파일: BGF리테일_2024.pdf
2024-12-02 20:43:54,456 - INFO - [Page 1] - 총 2개 청크 생성됨
2024-12-02 20:43:54,456 - INFO - Chunk 1: 길이 979 문자
2024-12-02 20:43:54,456 - INFO - Chunk 2: 길이 760 문자
2024-12-02 20:43:54,457 - INFO - --------------------------------------------------------------------------------
2024-12-02 20:43:54,809 - INFO - [Page 2] - 총 2개 청크 생성됨
2024-12-02 20:43:54,809 - INFO - Chunk 1: 길이 995 문자
2024-12-02 20:43:54,810 - INFO - Chunk 2: 길이 996 문자
2024-12-02 20:43:54,810 - INFO - --------------------------------------------------------------------------------
2024-12-02 20:43:55,155 - INFO - [Page 3] - 총 6개 청크 생성됨
2024-12-02 20:43:55,155 - INFO - Chunk 1: 길이 995 문자
2024-12-02 20:43:55,156 - INFO - Chunk 2: 길이 964 문자
2024-12-02 20:43:55,156 - INFO - Chunk 3: 길이 992 문자
2024-12-02 20:43:55,156 - INFO - Chunk 4: 길이 997 문자
2024-12-02 20:43:55,156 - INFO - Chunk 5: 길이 995 문자
2024-12-02 20:43:55